<a href="https://colab.research.google.com/github/Amartya-pixel/Cousera-test/blob/master/cs6910_assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import numpy as np
import pandas as pd

In [22]:
#Broadcasting
#suma along row and column
A = np.array([[10,20,30],[1,2,3],[11,21,31]])
B = np.array([[1],[2],[3]])
Sr = np.sum(A,0)
Sc = np.sum(A,1)
print(Sr)
print(Sc)
print(A/Sr)
print(A/Sc)
A + B

[22 43 64]
[60  6 63]
[[0.45454545 0.46511628 0.46875   ]
 [0.04545455 0.04651163 0.046875  ]
 [0.5        0.48837209 0.484375  ]]
[[0.16666667 3.33333333 0.47619048]
 [0.01666667 0.33333333 0.04761905]
 [0.18333333 3.5        0.49206349]]


array([[11, 21, 31],
       [ 3,  4,  5],
       [14, 24, 34]])

In [23]:
from keras.datasets import fashion_mnist

In [24]:
(x_train,y_train), (x_test, y_test) = fashion_mnist.load_data()

In [ ]:
# pd.DataFrame(x_train[0])
# pd.DataFrame(x_train.reshape(60000,28*28).transpose()[:,0].reshape(28,28))
# pd.DataFrame(x_train)
# pd.DataFrame(y_train)
# pd.DataFrame(x_test)
# pd.DataFrame(y_test)
# pd.DataFrame(outputs[1][-1])

In [25]:
X_train = np.float128(x_train.reshape(x_train.shape[0],x_train.shape[1]*x_train.shape[2]).T)
Y_train = np.float128(pd.get_dummies(y_train).T)
X_test = np.float128(x_test.reshape(x_test.shape[0],x_test.shape[1]*x_test.shape[2]).T)
Y_test = np.float128(pd.get_dummies(y_test).T)

In [26]:
def sigmoid(x):
  return 1.0 / (1.0 + np.exp(-x))
def tanh(x):
  return np.tanh(x)

In [27]:
def d_sigmoid(x):
  return sigmoid(x)*(1.0 - sigmoid(x))
def d_tanh(x):
  return 1.0 - sigmoid(x)*sigmoid(x) 

In [28]:
def softmax(x):
  e = np.exp(x)
  return e / np.sum(e,axis=0)

In [29]:
def initialize_parameters(n,layers):
  L = len(layers)
  biases = []
  weights = []
  for i in range(L):
    bias = np.float128(np.zeros((layers[i],1)))
    if i == 0:
      weight = np.float128(np.random.randn(layers[i],n))
    else:
      weight = np.float128(np.random.randn(layers[i],layers[i-1]))
    biases.append(bias)
    weights.append(weight)
  return (np.array(weights),np.array(biases))

In [30]:
def update_theta(theta,d_theta,learning_rate):
  weights, biases = theta
  d_weights,d_biases = d_theta
  weights = weights - learning_rate*d_weights
  biases = biases - learning_rate*d_biases
  return (weights,biases)


In [31]:
def update_theta_momentum(theta,d_theta,learning_rate,gamma,prev_weight,prev_bias):
  weights, biases = theta
  d_weights,d_biases = d_theta
  v_weight=gamma*prev_weight+learning_rate*d_weights
  v_bias=gamma*prev_bias+learning_rate*d_biases
  weights = weights - v_weight
  biases = biases - v_bias
  return (weights,biases,v_weight,v_bias)


In [32]:
def feedforward(X,theta):
  H = X
  L = len(theta[0])
  weights = theta[0]
  biases = theta[1]
  activations = []
  pre_activations = []
  for k in range(L-1):
    A = biases[k] + (weights[k] @ H)
    H = sigmoid(A)
    pre_activations.append(A)
    activations.append(H)
  AL = biases[L-1] + (weights[L-1] @ H)
  Y_hat = softmax(AL)
  pre_activations.append(AL)
  activations.append(Y_hat)
  return (np.array(pre_activations),np.array(activations))

In [33]:
def batch_gradient_descent(X,Y,inputs,theta,learning_rate):
  d_biases = []
  d_weights = []
  pre_activations , activations = inputs
  weights,biases = theta
  L = len(activations)
  Y_hat = activations[-1]
  d_AL = Y_hat - Y
  m = X.shape[1]
  for k in range(L-1,-1,-1):
    if(k == 0):
      d_W = (1/m)*(d_AL @ X.T)
      d_B = (1/m)*np.sum(d_AL,axis=1,keepdims=True)
    else:
      d_W = (1/m)*(d_AL @ activations[k-1].T)
      d_B = (1/m)*np.sum(d_AL,axis=1,keepdims=True)

      d_H = weights[k].T @ d_AL
      d_A = d_H*d_sigmoid(pre_activations[k-1])
      d_AL = d_A
    d_weights.insert(0,d_W)
    d_biases.insert(0,d_B)
  d_theta = (np.array(d_weights),np.array(d_biases))
  theta = update_theta(theta,d_theta,learning_rate)
  return theta 




In [34]:
def error(Y,inputs):
  Y_hat = inputs[1][-1]
  m = Y.shape[1]
  return (-1/m)*np.sum(Y*(np.log(Y_hat)))
  

In [35]:
def error_stochastic(Y,Y_hat):
  e = np.log(Y_hat)[np.argmax(Y)][0]
  return e

In [36]:
def stochastic_gradient_descent(X,Y,theta,learning_rate):
  m = X.shape[1]
  d_biases = []
  d_weights = []
  weights,biases = theta
  L = len(weights)
  # print(weights[0].shape,L)
  err = 0
  for i in range(m):
    pre_activations , activations = feedforward(X[:,i:i+1],theta)
    y_hat = activations[-1]
    d_al = y_hat - Y[:,i:i+1]
    err += error_stochastic(Y[:,i:i+1],y_hat)
    d_biases.clear()
    d_weights.clear()
    for k in range(L-1,-1,-1):
      # print(k)
      if(k == 0):
        # print(k)
        d_w = d_al @ X[:,i:i+1].T
        d_b = d_al
      else:
        d_w = d_al @ activations[k-1].T
        d_b = d_al
        # print(k,weights[k].shape,d_al.shape)
        d_h = weights[k].T  @ d_al
        d_a = d_h*d_sigmoid(pre_activations[k-1])
        d_al = d_a
      d_weights.insert(0,d_w)
      d_biases.insert(0,d_b)
    d_theta = (np.array(d_weights),np.array(d_biases))
    theta = update_theta(theta,d_theta,learning_rate)
    weights,biases = theta
  return (theta,err*(-1/m))

In [37]:
def backprop(X,Y,inputs,theta,batch_size):
  d_biases = []
  d_weights = []
  pre_activations , activations = inputs
  weights,biases = theta
  L = len(weights)
  Y_hat = activations[-1]
  d_AL = Y_hat - Y
  for k in range(L-1,-1,-1):
    if(k == 0):
      d_W = (1/batch_size)*(d_AL @ X.T)
      d_B = (1/batch_size)*np.sum(d_AL,axis=1,keepdims=True)
    else:
      d_W = (1/batch_size)*(d_AL @ activations[k-1].T)
      d_B = (1/batch_size)*np.sum(d_AL,axis=1,keepdims=True)

      d_H = weights[k].T @ d_AL
      d_A = d_H*d_sigmoid(pre_activations[k-1])
      d_AL = d_A
    d_weights.insert(0,d_W)
    d_biases.insert(0,d_B)
  d_theta = (np.array(d_weights),np.array(d_biases))
  return d_theta

In [38]:
def mini_batch_gradient_descent(X,Y,theta,learning_rate,batch_size):
  m = X.shape[1]
  L = len(weights)
  # print(weights[0].shape,L)
  err = 0
  for i in range(0,m//batch_size):
    start = i*batch_size
    stop = (i+1)*batch_size
    inputs = feedforward(X[:,start:stop],theta)
    err += error(Y[:,start:stop],inputs)
    d_theta = backprop(X[:,start:stop],Y[:,start:stop],theta,batch_size)
    theta = update_theta(theta,d_theta,learning_rate)
    weights,biases = theta
  if(m % batch_size != 0):
    start = i*batch_size
    stop = i+ (m%batch_size)

    inputs = feedforward(X[:,start:stop],theta)
    d_theta = backprop(X[:,start:stop],Y[:,start:stop],theta,m%batch_size)

    theta = update_theta(theta,d_theta,learning_rate)
    err+= error(Y[:,start:stop],inputs)
  return (theta,err)

In [47]:
def momentum_based_gradient_descent(X,Y,theta,learning_rate):
  m = X.shape[1]
  d_biases = []
  d_weights = []
  weights,biases = theta
  L = len(weights)
  err = 0
  prev_weights,prev_bias,gamma=0,0,0.9
  for i in range(m):
    pre_activations , activations = feedforward(X[:,i:i+1],theta)
    y_hat = activations[-1]
    d_al = y_hat - Y[:,i:i+1]
    err += error_stochastic(Y[:,i:i+1],y_hat)
    d_biases.clear()
    d_weights.clear()
    for k in range(L-1,-1,-1):
      # print(k)
      if(k == 0):
        # print(k)
        d_w = d_al @ X[:,i:i+1].T
        d_b = d_al
      else:
        d_w = d_al @ activations[k-1].T
        d_b = d_al
        # print(k,weights[k].shape,d_al.shape)
        d_h = weights[k].T  @ d_al
        d_a = d_h*d_sigmoid(pre_activations[k-1])
        d_al = d_a
      d_weights.insert(0,d_w)
      d_biases.insert(0,d_b)
    d_theta = (np.array(d_weights),np.array(d_biases))
    
    weights,biases,prev_weights,prev_bias= update_theta_momentum(theta,d_theta,learning_rate,gamma,prev_weights,prev_bias)
    theta=weights,biases
  return (theta,err*(-1/m))




In [ ]:
#nestrov

In [40]:
X_min = X_train
Y_min = Y_train

In [41]:
def model(epochs=1,layers=[],learning_rate=0.01):
  theta = initialize_parameters(X_min.shape[0],layers + [Y_min.shape[0]])
  for i in range(epochs):
    # outputs = feedforward_batch(X_min,theta)
    # theta = batch_gradient_descent(X_min,Y_min,outputs,theta,learning_rate)
    theta,err = stochastic_gradient_descent(X_min,Y_min,theta,learning_rate)
    print(err)
  return theta  

In [45]:
def model2(epochs=1,layers=[],learning_rate=0.01):
  theta = initialize_parameters(X_min.shape[0],layers + [Y_min.shape[0]])
  for i in range(epochs):
    # outputs = feedforward_batch(X_min,theta)
    # theta = batch_gradient_descent(X_min,Y_min,outputs,theta,learning_rate)
    theta,err = momentum_based_gradient_descent(X_min,Y_min,theta,learning_rate)
    print(err)
  return theta  

In [48]:
theta = model(5,[64,64,64],0.01)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the 

KeyboardInterrupt: ignored

In [49]:
theta = model2(5,[64,64,64],0.01)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the 

2.0874317706615380004
2.346262098803431431
2.2773796607650078144
2.2908505400795042936
2.2517575852905152146


In [43]:
weights,biases = theta

In [44]:
biases[3].shape

(10, 1)

In [ ]:
outputs = feedforward(X_test[:,:10],theta)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
pd.DataFrame(np.argmax(outputs[1][-1],0))

,0
0,9
1,4
2,1
3,1
4,4
5,4
6,4
7,4
8,7
9,5


In [ ]:
pd.DataFrame(np.argmax(Y_test[:,:10],0))

,0
0,9
1,2
2,1
3,1
4,6
5,1
6,4
7,6
8,5
9,7


In [ ]:
Y_pred = outputs[1][-1]
# pd.DataFrame([np.argmax(Y_test[:,:10],0)])
pd.DataFrame(np.log(Y_pred)*Y_test[:,:10])

,0,1,2,3,4,5,6,7,8,9
0,-0.00000,-0.000000,-0.000000,-0.000000,-0.00000,-0.000000,-0.00000,-0.00000,-0.00000,-0.000000
1,-0.00000,-0.000000,-0.624539,-0.624539,-0.00000,-3.955022,-0.00000,-0.00000,-0.00000,-0.000000
2,-0.00000,-1.298054,-0.000000,-0.000000,-0.00000,-0.000000,-0.00000,-0.00000,-0.00000,-0.000000
3,-0.00000,-0.000000,-0.000000,-0.000000,-0.00000,-0.000000,-0.00000,-0.00000,-0.00000,-0.000000
4,-0.00000,-0.000000,-0.000000,-0.000000,-0.00000,-0.000000,-1.12467,-0.00000,-0.00000,-0.000000
5,-0.00000,-0.000000,-0.000000,-0.000000,-0.00000,-0.000000,-0.00000,-0.00000,-1.30737,-0.000000
6,-0.00000,-0.000000,-0.000000,-0.000000,-1.65289,-0.000000,-0.00000,-1.65289,-0.00000,-0.000000
7,-0.00000,-0.000000,-0.000000,-0.000000,-0.00000,-0.000000,-0.00000,-0.00000,-0.00000,-1.350163
8,-0.00000,-0.000000,-0.000000,-0.000000,-0.00000,-0.000000,-0.00000,-0.00000,-0.00000,-0.000000
9,-1.09957,-0.000000,-0.000000,-0.000000,-0.00000,-0.000000,-0.00000,-0.00000,-0.00000,-0.000000


,0
0,-1.099570
1,-1.298054
2,-0.624539
3,-0.624539
4,-1.652890
5,-3.955022
6,-1.124670
7,-1.652890
8,-1.307370
9,-1.350163


In [ ]:
pd.DataFrame(Y_test)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,9960,9961,9962,9963,9964,9965,9966,9967,9968,9969,9970,9971,9972,9973,9974,9975,9976,9977,9978,9979,9980,9981,9982,9983,9984,9985,9986,9987,9988,9989,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
X = np.eye(10)
Y = np.ones(100).reshape(10,10)
X[:,0:1] @ Y[:,3:4].T

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
for k in range(4,-1,-1):
  print(k)
np.zeros((2,1))

4
3
2
1
0


array([[0.],
       [0.]])

In [ ]:
Y = np.zeros((10,1))
Y_hat = np.random.randn(10,1)
Y[3][0] = 1
Y*Y_hat

array([[ 0.        ],
       [-0.        ],
       [-0.        ],
       [ 0.88613674],
       [-0.        ],
       [-0.        ],
       [ 0.        ],
       [ 0.        ],
       [-0.        ],
       [ 0.        ]])

In [ ]:
np.ceil(6000/64)

94.0